# Validation of takeout results

### Setup

In [1]:
# Import packages
import os
import json
import statsapi
import datetime
from datetime import timedelta, datetime
import pandas as pd

from agno.agent import Agent
from agno.models.groq import Groq
from agno.tools import Toolkit

In [2]:
api_key = os.getenv('GROQ_API_KEY')

We will configure multiple LLMs using [Agno Agents](https://github.com/agno-agi/agno), each requiring a Groq API Key for access which you can create [here](https://console.groq.com/keys). These models include different versions of Meta's LLaMA 3 and other specialized models like Google's Gemma 2 and Mixtral. Each model will be used by different agents to generate diverse outputs for the MLB game recap.

In [3]:
llm_model = Groq(id="llama-3.3-70b-versatile", api_key=api_key)

In [4]:
import pandas as pd
grubhub = pd.read_csv('grubhub.csv', 
                      names=['na','restaurant','price','date'])
grubhub.drop('na',axis=1,inplace=True)
grubhub['date'] = pd.to_datetime(grubhub['date'])
grubhub

,restaurant,price,date
0,Little Caesars Pizza,21.24,2025-01-18
1,Five Guys,32.93,2025-01-05
2,Maiwand Kabob,53.43,2024-12-29
3,Chipotle,51.06,2024-12-27
4,Five Guys,32.93,2024-12-21
...,...,...,...
56,Maiwand Kabob,60.63,2024-02-06
57,An Loi,47.42,2024-02-01
58,Soul Boat,34.68,2024-01-22
59,Bonchon,25.83,2024-01-12


### Define Tools

In [5]:
def helper(restaurant_name: str, start_date: str, end_date: str) -> pd.DataFrame:
    name_match = True if not restaurant_name else grubhub['restaurant'] == restaurant_name
    date_match = (grubhub['date'] >= start_date) & (grubhub['date'] <= end_date)
    matches = grubhub[name_match & date_match]

    return matches

In [6]:
def get_frequency(restaurant_name: str = '', start_date: str = '2000-01-01', end_date: str = '2026-01-01') -> str:
    """Gets the number of times a restaurant appears in the data
    on or inbetween the given dates
    """
    matches = helper(restaurant_name, start_date, end_date)
    retval= {
        'frequency': matches.shape[0]
    }
    return json.dumps(retval)

print(get_frequency(start_date='2024-02-01', end_date='2024-02-28', restaurant_name='An Loi'))
print(get_frequency(start_date='2024-02-01', end_date='2024-02-28'))


{"frequency": 2}
{"frequency": 6}


In [7]:
def get_cost(restaurant_name: str = '', start_date: str = '2000-01-01', end_date: str = '2026-01-01') -> str:
    """Get the total cost of meals from a restaurant within a date range
    """
    matches = helper(restaurant_name, start_date, end_date)
    
    retval = {
        'total_cost': matches['price'].sum()
    }
    return json.dumps(retval)

print(get_cost(start_date='2024-02-01', end_date='2024-02-28', restaurant_name='An Loi'))


{"total_cost": 74.07}


### Define Agents

In [8]:
takeout_q = Agent(
    name = 'takeout_q',
    model=llm_model,
    description="An agent that answers questions about takeout history",
    instructions = [
        f"Parse the restaurant name (use '' if user does not specify), start date, and end date from the user question.",
        "Use the tools to answer questions about my takeout habits",    
    ],
    tools=[get_frequency, get_cost],
    show_tool_calls=True
)

class TakeoutTools(Toolkit):
    def __init__(self):
        super().__init__(name="takeout_tools")
        setattr(self.__class__, 'get_frequency', classmethod(get_frequency))
        setattr(self.__class__, 'get_cost', classmethod(get_cost))
        self.register(self.get_frequency)
        self.register(self.get_cost)

toolkit_takeout_q = Agent(
    name = 'toolkit_takeout_q',
    model=llm_model,
    description="An agent that answers questions about takeout history",
    instructions = [
        f"Parse the restaurant name (use '' if user does not specify), start date as yyyy-MM-dd, and end date as yyyy-MM-dd from the user question.",
        "Use the available tools to answer questions about takeout history",    
    ],
    tools=[TakeoutTools()],
    show_tool_calls=True,
    markdown=False
)


In [9]:
user_prompt = {
    0: ['how many times did I eat at An Loi during the month of August 2024?',
        get_frequency(restaurant_name='An Loi', start_date='2024-08-01', end_date='2024-08-31')],
    
    1: ['how many times did I order takeout during the month of August 2024?',
        get_frequency(restaurant_name='', start_date='2024-08-01', end_date='2024-08-31')],
    
    2: ['how much did I spend at An Loi during the month of June 2024?',
        get_cost(restaurant_name='An Loi', start_date='2024-06-01', end_date='2024-06-30')],
    
    3: ['how much did I spend on takeout in July of 2024?',
        get_cost(restaurant_name='', start_date='2024-07-01', end_date='2024-07-31')],
    
    4: ['did I spend more in takeout in October or November of 2024?',
        'October' if get_cost(restaurant_name='', start_date='2024-10-01', end_date='2024-10-31') > get_cost(restaurant_name='', start_date='2024-10-01', end_date='2024-10-31') else 'November'],
    
    5: ['how much did I spend on takeout in January 2024 and February 2024?',
        get_cost(restaurant_name='', start_date='2024-01-01', end_date='2024-02-29')],
    
    6: ['how much did I spend on takeout in January 2024?',
        get_cost(restaurant_name='', start_date='2024-01-01', end_date='2024-01-31')],

    7: ['how many times did I eat at An Loi during the month of February 2024?',
        get_frequency(restaurant_name='An Loi', start_date='2024-02-01', end_date='2024-02-29')],
    
    8:['how many times did I eat out during the month of February 2024?',
        get_frequency(restaurant_name='', start_date='2024-02-01', end_date='2024-02-29')],    
}
user_prompt

{0: ['how many times did I eat at An Loi during the month of August 2024?',
  '{"frequency": 1}'],
 1: ['how many times did I order takeout during the month of August 2024?',
  '{"frequency": 5}'],
 2: ['how much did I spend at An Loi during the month of June 2024?',
  '{"total_cost": 45.4}'],
 3: ['how much did I spend on takeout in July of 2024?',
  '{"total_cost": 95.47}'],
 4: ['did I spend more in takeout in October or November of 2024?',
  'November'],
 5: ['how much did I spend on takeout in January 2024 and February 2024?',
  '{"total_cost": 328.90000000000003}'],
 6: ['how much did I spend on takeout in January 2024?',
  '{"total_cost": 101.32}'],
 7: ['how many times did I eat at An Loi during the month of February 2024?',
  '{"frequency": 2}'],
 8: ['how many times did I eat out during the month of February 2024?',
  '{"frequency": 6}']}

In [12]:
%%time
result = takeout_q.run(user_prompt[5][0], stream=False)
print(result.content)

 - Running: get_cost(start_date=2024-01-01, end_date=2024-02-28, restaurant_name=)

You spent a total of $328.90 on takeout in January and February 2024.
CPU times: user 79.4 ms, sys: 16.6 ms, total: 96 ms
Wall time: 1.22 s


In [14]:
from ast import literal_eval


agents = [takeout_q, toolkit_takeout_q]

cols = ['agent', 'prompt_id', 'iteration', 'success']
metrics = []

for agent in agents:
    for prompt_id in [2,3,5]: #range(1): #range(len(user_prompt)):
        print(f'{prompt_id=}')
        for it in range(5):
            result = agent.run(user_prompt[prompt_id][0], stream=False)
            # check result
            dict_expected = literal_eval(user_prompt[prompt_id][1])
            expected = list(dict_expected.values())[0]
            metric = {
                'agent': agent.name,
                'prompt_id': prompt_id,
                'iteration': it,
                'success': str(expected) in result.content,
                'content' : result.content
            }
            metrics.append(metric)

telem = pd.DataFrame.from_dict(metrics)
pd.set_option('max_colwidth', None)
telem

prompt_id=2
prompt_id=3
prompt_id=5
prompt_id=2


WARNING  Could not run function get_cost(start_date=2024-06-01, end_date=2024-06-30, restaurant=An Loi)

ERROR    1 validation error for get_cost                                                                           
         restaurant                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value='An Loi', input_type=str]    
             For further information visit https://errors.pydantic.dev/2.10/v/unexpected_keyword_argument          
         Traceback (most recent call last):                                                                        
           File                                                                                                    
         "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.10/site-packages/agno/tools/fu
         nction.py", line 363, in execute                                                                          
             self.result = self.function.entrypoint(**entrypoint_args, **self.arguments)                           
           File                                                                                                    
         "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.10/site-packages/pydantic/_int
         ernal/_validate_call.py", line 38, in wrapper_function                                                    
             return wrapper(*args, **kwargs)                                                                       
           File                                                                                                    
         "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.10/site-packages/pydantic/_int
         ernal/_validate_call.py", line 111, in __call__                                                           
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
         pydantic_core._pydantic_core.ValidationError: 1 validation error for get_cost                             
         restaurant                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value='An Loi', input_type=str]    
             For further information visit https://errors.pydantic.dev/2.10/v/unexpected_keyword_argument

WARNING  Could not run function get_cost(start_date=2024-06-01, end_date=2024-06-30, restaurant=An Loi)

ERROR    1 validation error for get_cost                                                                           
         restaurant                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value='An Loi', input_type=str]    
             For further information visit https://errors.pydantic.dev/2.10/v/unexpected_keyword_argument          
         Traceback (most recent call last):                                                                        
           File                                                                                                    
         "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.10/site-packages/agno/tools/fu
         nction.py", line 363, in execute                                                                          
             self.result = self.function.entrypoint(**entrypoint_args, **self.arguments)                           
           File                                                                                                    
         "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.10/site-packages/pydantic/_int
         ernal/_validate_call.py", line 38, in wrapper_function                                                    
             return wrapper(*args, **kwargs)                                                                       
           File                                                                                                    
         "/Users/robertholder/Documents/installs/miniconda3/envs/agnoenv/lib/python3.10/site-packages/pydantic/_int
         ernal/_validate_call.py", line 111, in __call__                                                           
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
         pydantic_core._pydantic_core.ValidationError: 1 validation error for get_cost                             
         restaurant                                                                                                
           Unexpected keyword argument [type=unexpected_keyword_argument, input_value='An Loi', input_type=str]    
             For further information visit https://errors.pydantic.dev/2.10/v/unexpected_keyword_argument

prompt_id=3
prompt_id=5


,agent,prompt_id,iteration,success,content
0,takeout_q,2,0,False,"<function=get_cost{""restaurant_name"": ""An Loi"", ""start_date"": ""2024-06-01"", ""end_date"": ""2024-06-30""}</function>"
1,takeout_q,2,1,True,"- Running: get_cost(restaurant_name=An Loi, start_date=2024-06-01, end_date=2024-06-30)\n\nYou spent $45.4 at An Loi during the month of June 2024."
2,takeout_q,2,2,True,"- Running: get_cost(restaurant_name=An Loi, start_date=2024-06-01, end_date=2024-06-30)\n\nYou spent a total of $45.4 at An Loi during the month of June 2024."
3,takeout_q,2,3,True,"- Running: get_cost(restaurant_name=An Loi, start_date=2024-06-01, end_date=2024-06-30)\n\nYou spent $45.4 at An Loi during the month of June 2024."
4,takeout_q,2,4,True,"- Running: get_cost(start_date=2024-06-01, end_date=2024-06-30, restaurant_name=An Loi)\n\nYou spent $45.4 at An Loi during the month of June 2024."
5,takeout_q,3,0,True,"- Running: get_cost(restaurant_name=, start_date=2024-07-01, end_date=2024-07-31)\n\nYou spent $95.47 on takeout in July 2024."
6,takeout_q,3,1,False,"<function=get_cost={""restaurant_name"": """", ""start_date"": ""2024-07-01"", ""end_date"": ""2024-07-31""}</function>"
7,takeout_q,3,2,True,"- Running: get_cost(start_date=2024-07-01, end_date=2024-07-31, restaurant_name=)\n\nYou spent $95.47 on takeout in July 2024."
8,takeout_q,3,3,True,"- Running: get_cost(start_date=2024-07-01, end_date=2024-07-31, restaurant_name=)\n\nYou spent a total of $95.47 on takeout in July 2024."
9,takeout_q,3,4,True,"- Running: get_cost(start_date=2024-07-01, end_date=2024-07-31, restaurant_name=)\n\nYou spent $95.47 on takeout in July 2024."
